<a href="https://colab.research.google.com/github/cathyxinchangli/ATMS-597-SP-2020/blob/sarah/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# TBA 
# (I'm not very good with code documentation e.g. common practices etc. so any help is appreciated! - XL)


import requests
import pandas as pd

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    ---
    S. Moline, 'Hands on Data Analysis with Pandas'
    https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/blob/master/ch_04/0-weather_data_collection.ipynb
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'HfZphTTGbXckpmEuqevAXQZFRCTEJgkJ' # obtained by X. Li, 2/4/2020
        },
        params=payload
    )


In [0]:
# documentation of GHCN (Global Historical Climatology Network) – Daily Documentation: 
# https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf

# getting station information (not sure if this is necessary)
!wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

--2020-02-05 21:29:12--  https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt
Resolving www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)... 205.167.25.178, 205.167.25.171, 205.167.25.172, ...
Connecting to www1.ncdc.noaa.gov (www1.ncdc.noaa.gov)|205.167.25.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9896966 (9.4M) [text/plain]
Saving to: ‘ghcnd-stations.txt’

ghcnd-stations.txt  100%[===================>]   9.44M  4.40MB/s    in 2.1s    

2020-02-05 21:29:15 (4.40 MB/s) - ‘ghcnd-stations.txt’ saved [9896966/9896966]



In [0]:
!head -20 ghcnd-stations.txt

ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       
ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    
AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196
AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194
AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217
AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218
AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930
AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938
AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948
AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                       40990
AG000060390  36.7167    3.2500   24.0    ALGER-DAR EL BEIDA             GSN     60390
AG000060590  30.5667    2.8667  397.0    EL-GOLEA     

In [0]:
stations = pd.read_fwf('ghcnd-stations.txt',names=['ID','LAT [deg]','LON [deg]','ELV [m]','NAME','col5','col6','col7']) # I don't know what the last three columns are...
stations

,ID,LAT [deg],LON [deg],ELV [m],NAME,col5,col6,col7
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,NaN,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,NaN,NaN,41217.0
...,...,...,...,...,...,...,...,...
115076,ZI000067969,21.0500,29.3670,861.0,WEST NICHOLSON,NaN,NaN,67969.0
115077,ZI000067975,20.0670,30.8670,1095.0,MASVINGO,NaN,NaN,67975.0
115078,ZI000067977,21.0170,31.5830,430.0,BUFFALO RANGE,NaN,NaN,67977.0
115079,ZI000067983,20.2000,32.6160,1132.0,CHIPINGE,NaN,GSN,67983.0


In [0]:
stations['ELV [m]'].describe()

count    115081.000000
mean        447.756902
std         667.260086
min        -999.900000
25%          80.000000
50%         256.000000
75%         610.000000
max        5033.000000
Name: ELV [m], dtype: float64

In [0]:
stations[:][stations['ELV [m]'] > 4000]

,ID,LAT [deg],LON [deg],ELV [m],NAME,col5,col6,col7
17498,AYM00089577,80.3700,77.3700,4084.0,DOME PLATEAU DOME A,NaN,GSN,89577.0
17651,BLM00085201,16.5130,-68.1920,4061.5,EL ALTO INTL,NaN,NaN,85201.0
32718,CHM00055228,32.5000,80.0830,4280.0,SHIQUANHE,NaN,NaN,55228.0
32719,CHM00055279,31.3670,90.0170,4701.0,BAINGOIN,NaN,NaN,55279.0
32720,CHM00055299,31.4830,92.0670,4508.0,NAGQU,NaN,NaN,55299.0
32721,CHM00055472,30.9500,88.6330,4670.0,XAINZA,NaN,NaN,55472.0
32724,CHM00056004,34.2170,92.4330,4535.0,TUOTUOHE,NaN,NaN,56004.0
32725,CHM00056021,34.1330,95.7830,4176.0,QUMARLEB,NaN,NaN,56021.0
32727,CHM00056033,34.9170,98.2170,4273.0,MADOI,NaN,NaN,56033.0
32732,CHM00056106,31.8830,93.7830,4024.0,SOG XIAN,NaN,NaN,56106.0


## Collect All Datapoints
I believe we have to go through a ton of years correct?


In [0]:
import datetime

from IPython import display # for updating the cell dynamically

current = datetime.date(1998, 1, 1)
end = datetime.date(2019, 1, 1)

results = []

while current < end:
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'locationid' : 'CITY:US360019', # NYC
            'startdate' : current,
            'enddate' : current,
            'units' : 'metric',
            'limit' : 1000 # max allowed
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'])

    # update the current date to avoid an infinite loop
    current += datetime.timedelta(days=1)

'Gathering data for 1998-09-12'

KeyboardInterrupt: ignored

--2020-02-05 22:02:30--  https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CITY
Resolving www.ncdc.noaa.gov (www.ncdc.noaa.gov)... 205.167.25.177, 205.167.25.178, 205.167.25.171, ...
Connecting to www.ncdc.noaa.gov (www.ncdc.noaa.gov)|205.167.25.177|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-02-05 22:02:31 ERROR 400: Bad Request.

